In [ ]:
import numpy as np 
import pandas as pd
import os

H_leaf_dir = os.path.join('../input/leaves-healthy-or-diseased/leaves_healthy_or_diseased/healthy')
D_leaf_dir = os.path.join('../input/leaves-healthy-or-diseased/leaves_healthy_or_diseased/diseased')

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
image_size = 220
batch_size = 32


target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory(
        '../input/leaves-healthy-or-diseased/leaves_healthy_or_diseased/', 
        target_size=(200, 200),
        batch_size = batch_size,
        
        classes = ['healthy','diseased'],
       
        class_mode='categorical')

In [ ]:
for image_batch, label_batch in train_generator:
  break
  image_batch.shape, label_batch.shape

In [ ]:
print(train_generator.class_indices)

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 200x 200 with 3 bytes color
    # The first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a dense layer
    tf.keras.layers.Flatten(),
    # 128 neuron in the fully-connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    # 5 output neurons for 2 classes with the softmax activation
    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])

In [ ]:
total_sample=train_generator.n

In [ ]:
n_epochs = 3

In [ ]:
history = model.fit(
        train_generator, 
        steps_per_epoch=int(total_sample/batch_size),  
        epochs=n_epochs,
        verbose=1)

In [ ]:
# from keras.layers import Activation, Dense
# model.add(Dense(3, activation='softmax'))

In [ ]:
model.save('HD_Model.h5')

In [ ]:
model.save(filepath="save_model/")

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('save_model')
tflite_model = converter.convert()
open("leaves_model.tflite", "wb").write(tflite_model)

In [ ]:
# import os
# os.remove("./leaves_model.tflite")

In [ ]:
MODEL_NAME = model
def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")

In [ ]:
export_model(tf.compat.v1.train.Saver(), model,["conv2d_1_input"], "dense_2/Softmax"H)
# tf.compat.v1.train.Saver()

In [ ]:
from tensorflow.keras import Sequential
model.export('leaves_model.tflite', 'leavesLabels.txt')
#  model.export(export_dir='./',tflite_filename='leaves_model.tflite', label_filename='leavesLabels.txt')